In [65]:
#!pip install -q pyomo
#!apt-get install -y -qq coinor-cbc

**Q1)**

The given problem can be modelled as follows:\

Let r = [3,2,6,5,5,4,6,8,8,5,4]

$ \text{Decision  variables} : c_i  \; \qquad i = 0,1,2,3....10$ \
Where $c_i$ denoted registration in course Ci ($c_0$ denotes registration in course C0 etc)\


$ \text{Maximize} \sum_{i=0}^{10} c_ir_i$

S.T\
\begin{align}
c_2 + c_7 \leq 1, \\
c_4 + c_9 \leq 1, \\
c_2 + c_8 \leq 1, \\
c_0 + c_1 + c_2 \leq 2,\\
c_5 + c_6 \leq 1,\\
c_3+ c_6+ c_9 \leq 2,\\
\sum_{i=0}^{10} c_i \leq 9,\\
c_{10} - c_0 \leq 0, \\
c_0 + c_1 + c_2 \geq 1,\\
\sum_{i=0}^{10} c_ir_i \geq 30,\\
\sum_{i=0}^{10} c_i \geq 5,\\
c_i \in \{0,1\} \qquad i = 0,1,2 ... 10 
\end{align}

Where $c_i = 1$ if course Ci is registered for , and $c_i = 0$ otherwise\
And $r_i$ is the $(i+1)^{th}$ element of r


All decision variables are integers

In [66]:
from pyomo.environ import *
import numpy as np
cbc = SolverFactory('cbc')

In [67]:
def fx(*coefs):
  lst = [0]*11
  for i in coefs:
    lst[i]=1
  return lst  

In [68]:
n = 11
m = 11
r = np.array([3,2,6,5,5,4,6,8,8,5,4])

leq_cons_coef = np.array([fx(2,7), fx(4,9), fx(2,8), fx(0,1,2),fx(5,6),fx(3,6,9),fx(*range(11)),[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])
geq_cons_coef = np.array([fx(0,1,2),r,fx(*range(11))])

leq_consrhs = np.array([1,1,1,2,1,2,9,0])
geq_consrhs = np.array([1,30,5])

rows = np.arange(m)
cols=np.arange(n)

In [69]:
m2 = ConcreteModel()

m2.c = Var(cols, domain=Binary)
m2.cons = ConstraintList()


In [70]:
for i in range(8):
  m2.cons.add(sum(leq_cons_coef[i][j]*m2.c[j] for j in cols)<=leq_consrhs[i])

for i in range(3):
  m2.cons.add(sum(geq_cons_coef[i][j]*m2.c[j] for j in cols)>=geq_consrhs[i])
 

In [71]:
m2.credits = Objective(expr=summation(r,m2.c),sense=maximize)

In [72]:
opt = cbc.solve(m2)
#opt.write()

**Q3)**

In [73]:
print('The optimal value of the objective function is:',m2.credits())
for i in range(11):
  print('The value of c%d is'%i,m2.c[i]())

The optimal value of the objective function is: 41.0
The value of c0 is 1.0
The value of c1 is 1.0
The value of c2 is 0.0
The value of c3 is 1.0
The value of c4 is 1.0
The value of c5 is 0.0
The value of c6 is 1.0
The value of c7 is 1.0
The value of c8 is 1.0
The value of c9 is 0.0
The value of c10 is 1.0


In [74]:
m2a = m2.clone()
m2a.c.domain = PositiveReals
m2a.c.setub(1)



In [75]:
opta= cbc.solve(m2a)
#opta.write()

**Q4)**

In [76]:
print('The optimal value of the objective function is:',m2a.credits())
for i in range(11):
  print('The value of c%d is'%i,m2a.c[i]())

The optimal value of the objective function is: 41.0
The value of c0 is 1.0
The value of c1 is 1.0
The value of c2 is 0.0
The value of c3 is 1.0
The value of c4 is 1.0
The value of c5 is 0.0
The value of c6 is 1.0
The value of c7 is 1.0
The value of c8 is 1.0
The value of c9 is 0.0
The value of c10 is 1.0


**Q5)**

For this particular problem, solution to MILP and solution to LP is common.
Hence we can obtain solution by rounding off solution of LP.
However this is not true in general.

**Q6)**

The given problem can be modified as follows:\

Let r = [3,2,6,5,5,4,6,8,8,5,4,9]

$ \text{Decision  variables} : c_i  \; \qquad i = 0,1,2,3....11$ \
Where $c_i$ denoted registration in course Ci ($c_0$ denotes registration in course C0 etc)\


$ \text{Maximize} \sum_{i=0}^{11} c_ir_i$

S.T\
\begin{align}
c_2 + c_7 \leq 1, \\
c_4 + c_9 \leq 1, \\
c_2 + c_8 \leq 1, \\
c_0 + c_1 + c_2 \leq 2,\\
c_5 + c_6 \leq 1,\\
c_3+ c_6+ c_9 \leq 2,\\
c_{11} + c_7 \leq 1,\\
c_{11} + c_8 \leq 1, \\
c_{11} + c_9 \leq 1, \\
\sum_{i=0}^{11} c_i \leq 9,\\
c_{10} - c_0 \leq 0, \\
c_0 + c_1 + c_2 \geq 1,\\
\sum_{i=0}^{11} c_ir_i \geq 30,\\
\sum_{i=0}^{11} c_i \geq 5,\\
c_i \in \{0,1\} \qquad i = 0,1,2 ... 11 
\end{align}

Where $c_i = 1$ if course Ci is registered for , and $c_i = 0$ otherwise\
And $r_i$ is the $(i+1)^{th}$ element of r

**Q7)**

In [77]:
m2b = m2.clone()
m2b.c11 = Var(domain = Binary)
m2b.cons.add(m2b.c[7] + m2b.c11<=1)
m2b.cons.add(m2b.c[8] + m2b.c11<=1)
m2b.cons.add(m2b.c[9] + m2b.c11<=1)

m2b.cred2= Objective(expr=summation(r,m2b.c)+9*m2b.c11, sense=maximize)
m2b.credits.deactivate()

In [78]:
optb = cbc.solve(m2b)
#optb.write()

In [79]:
print('The optimal value of the objective function is:',m2b.cred2())
for i in range(11):
  print('The value of c%d is'%i,m2b.c[i]())

print('The value of c11 is',m2b.c11())



The optimal value of the objective function is: 41.0
The value of c0 is 1.0
The value of c1 is 1.0
The value of c2 is 0.0
The value of c3 is 1.0
The value of c4 is 1.0
The value of c5 is 0.0
The value of c6 is 1.0
The value of c7 is 1.0
The value of c8 is 1.0
The value of c9 is 0.0
The value of c10 is 1.0
The value of c11 is 0.0


In [80]:
m2c = m2b.clone()

In [81]:
m2c.c.domain = PositiveReals
m2c.c11.domain= PositiveReals


m2c.c.setub(1)
m2c.c11.setlb(0)
m2c.c11.setub(1)

In [82]:
optc = cbc.solve(m2c)
#optc.write()

**Q8)**

In [83]:
print('The optimal value of the objective function is:',m2c.cred2())
for i in range(11):
  print('The value of c%d is'%i,m2c.c[i]())
print('The value of c11 is',m2c.c11())
    

The optimal value of the objective function is: 41.0
The value of c0 is 1.0
The value of c1 is 1.0
The value of c2 is 0.0
The value of c3 is 1.0
The value of c4 is 1.0
The value of c5 is 0.0
The value of c6 is 1.0
The value of c7 is 1.0
The value of c8 is 1.0
The value of c9 is 0.0
The value of c10 is 1.0
The value of c11 is 0.0


For this particular problem, solution to MILP and solution to LP is common.
Hence we can obtain solution by rounding off solution of LP.
However this is not true in general.